In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

# Load text datasets from Google Drive
path = "/content/drive/MyDrive/CSC8111_Coursework/" # Change path to run program

train = pd.read_csv(path + "Tweets_train.csv", encoding="ISO-8859-1")
test = pd.read_csv(path + "Tweets_test.csv", encoding="ISO-8859-1")
dev = pd.read_csv(path + "Tweets_dev.csv", encoding="ISO-8859-1")

train = train.to_numpy()
test = test.to_numpy()
dev = dev.to_numpy()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
combined_array = train
combined_array = np.r_[combined_array, test]
combined_array = np.r_[combined_array, dev]

In [ ]:
def create_arrays(array):
  array_x = []
  array_y = []

  count_pos = 0
  count_neu = 0
  count_neg = 0

  for value in array:
    array_x.append(value[1])
    array_y.append(value[2])
    if value[2] == "positive": count_pos += 1
    elif value[2] == "neutral": count_neu += 1
    elif value[2] == "negative": count_neg += 1

  print("Positive count: %d" % count_pos)
  print("Neutral count: %d" % count_neu)
  print("Negative count: %d" % count_neg)

  array_x = np.array(array_x, dtype=object)
  array_y = np.array(array_y, dtype=object)

  return array_x, array_y

In [ ]:
from nltk.sem.logic import Tokens
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
import string

def clean_array(array):
  clean_array = []
  for tweet in array:
    # Split tweet into words
    tokens = tweet.split()
    # Remove @ tags
    tokens = [word for word in tokens if word[0] != "@"]
    # Remove punctuation
    remove_table = str.maketrans("", "", string.punctuation)
    tokens = [word.translate(remove_table) for word in tokens]
    # Remove non-alphabetic words
    tokens = [word for word in tokens if word.isalpha()]
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if not word in stop_words]
    # Remove one character words
    tokens = [word for word in tokens if len(word) > 1]
    # Convert all characters to lower case
    tokens = [word.lower() for word in tokens]
    # Join words back together into single string
    clean_array.append(" ".join(tokens))
  clean_array = np.array(clean_array, dtype=object)
  
  return clean_array

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
combined_array_x, combined_array_y = create_arrays(combined_array)
combined_array_x = clean_array(combined_array_x)

Positive count: 2363
Neutral count: 3099
Negative count: 9178


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
vector_array_x = vectorizer.fit_transform(combined_array_x).toarray()

In [ ]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(vector_array_x, combined_array_y, test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=200)
model.fit(train_x, train_y)

RandomForestClassifier(n_estimators=200)

In [ ]:
predictions = model.predict(test_x)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(test_y,predictions))
print(classification_report(test_y,predictions))
print("Accuracy: %.2f" % (accuracy_score(test_y, predictions) * 100))

[[1686  110   48]
 [ 267  295   44]
 [ 150   57  271]]
              precision    recall  f1-score   support

    negative       0.80      0.91      0.85      1844
     neutral       0.64      0.49      0.55       606
    positive       0.75      0.57      0.64       478

    accuracy                           0.77      2928
   macro avg       0.73      0.66      0.68      2928
weighted avg       0.76      0.77      0.76      2928

Accuracy: 76.91
